In [167]:
import numpy as np 
import matplotlib.pyplot as plt
from mpmath import tanh

In [168]:
def apprentissage_batch_pas_dynamique(vecteurs, classifications, dimension_n, pas_alpha_initial, omega_initial):
    omega = omega_initial
    alpha = pas_alpha_initial
    nb_iterations = 0
    
    print("Début de l'apprentissage batch avec pas dynamique...")

    while True:
        
        delta_omega = np.zeros(dimension_n + 1)
        bien_classes = True

        for vecteur, classification in zip(vecteurs, classifications):
            vecteur_augmente = np.insert(vecteur, 0, 1) 
            prediction = np.sign(np.dot(omega, vecteur_augmente))
            
            if prediction != classification: 
                bien_classes = False
                delta_omega += alpha * (classification - prediction) * vecteur_augmente
    
        omega += delta_omega
        nb_iterations += 1

        # pas dynamique
        if nb_iterations % 100 == 0:
            alpha *= 0.5
            # print(f"Iteration {nb_iterations}: ajustement du pas, nouveau alpha = {alpha}")

        if bien_classes:  # si toutes les paires sont correctement classées
            break

    print("Toutes les paires ont été correctement classées.")
    print(f"Omega final : {omega}")
    print(f"Nombre d'itérations nécessaires : {nb_iterations}")

    return omega, nb_iterations

In [169]:
# fonction pour calculer la stabilité d'un vecteur
def stabilite(X, to, W):
    return (to * np.dot(W, np.insert(X, 0, 1)))/ np.linalg.norm(W)

# fonction pour calculer les stabilités
def liste_stabilites(vecteurs, classes, w_separateur):
    return np.array([stabilite(vecteurs[i], classes[i], w_separateur) for i in range(len(vecteurs))])

In [170]:
def erreur_minimerror(stabilite, beta):
    return 1 - tanh((beta*stabilite)/2)

def erreur_moyenne_minimerror_l(stabilites, beta):
    return 1/2 * np.sum(np.array([erreur_minimerror(stabilite, beta) for stabilite in stabilites]))

In [171]:
def minimerror_l(vecteurs, classes, w_separateur, dimension_n, T=0.5, max_iterations=1000, erreur_limite=0.0, epsilon=1e-1, delta_b0=1e-3):
    beta = 1 / T
    erreur_precedente = float('inf')
    max_beta = 1e3  # Limite supérieure pour beta
    min_cosh_value = 1e-7  # Valeur minimale pour stabiliser cosh
    max_cosh_input = 20  # Plafond pour l'entrée de cosh

    vecteurs = np.array(vecteurs, dtype=float)
    classes = np.array(classes, dtype=float)
    w_separateur = np.array(w_separateur, dtype=float)

    for iteration in range(max_iterations):
        stabilites = liste_stabilites(vecteurs, classes, w_separateur)
        delta_w = np.zeros(dimension_n + 1, dtype=float)
        erreur_actuelle = float(erreur_moyenne_minimerror_l(stabilites, beta))

        if erreur_actuelle <= erreur_limite:
            print(f"Erreur limite atteinte après {iteration} itérations.")
            return w_separateur, erreur_actuelle

        for u, gamma in enumerate(stabilites):
            gamma_beta = beta * gamma / 2
            if gamma_beta > max_cosh_input:
                cosh_value = 0.5 * np.exp(gamma_beta)  # Approximation pour cosh(x) lorsque x est grand
            else:
                cosh_value = max(np.cosh(gamma_beta), min_cosh_value)
            delta_w += -classes[u] * np.insert(vecteurs[u], 0, 1) / (cosh_value ** 2)

        w_separateur -= epsilon * (beta / 4) * delta_w
        w_separateur /= np.linalg.norm(w_separateur)

        beta = min(beta * (1 + delta_b0), max_beta)  # Limiter la croissance de beta

        if iteration % 100 == 0 or iteration == max_iterations - 1:
            print(f"Iteration {iteration}: Erreur = {erreur_actuelle:.6f}, Norme W = {np.linalg.norm(w_separateur):.4f}")

        if erreur_actuelle > erreur_precedente:
            epsilon *= 0.5
        erreur_precedente = erreur_actuelle

    print("Nombre maximal d'itérations atteint.")
    print(w_separateur)
    return w_separateur, erreur_actuelle

In [172]:
# On définit deux ensembles par exemple à séparer 

# or
or_vectors = np.array([[0,0],[0,1],[1,0],[1,1]])
or_classifications = np.array([-1,1,1,1])

# xor
xor_vectors = np.array([[0,0],[0,1],[1,0],[1,1]])
xor_classifications = np.array([1,-1,-1,1])

In [173]:
# On définit un omega initial
w_initial = np.random.uniform(low=-0.005, high=0.005, size=3)
w_initial

array([ 0.00447368,  0.0041919 , -0.00341209])

In [174]:
# on teste notre algorithme batch à pas dynamique
w, nb_iter = apprentissage_batch_pas_dynamique(or_vectors, or_classifications, 2, 0.1, w_initial)
print(f"w_separateur: {w}")
print(f"nombre itérations: {nb_iter}")

Début de l'apprentissage batch avec pas dynamique...
Toutes les paires ont été correctement classées.
Omega final : [-0.19552632  0.4041919   0.39658791]
Nombre d'itérations nécessaires : 6
w_separateur: [-0.19552632  0.4041919   0.39658791]
nombre itérations: 6


In [175]:
# fonction pour généraliser sur un ensemble
def classer_ensemble(vecteurs, w_separateur):
    return np.array([1 if np.dot(np.insert(vecteur, 0, 1), w_separateur) > 0 else -1 for vecteur in vecteurs])

In [176]:
def algorithme_monoplan(w_initial, vecteurs, classifications, pas_alpha_initial, dimension_n=2, max_iterations=100, max_hyperplans=2, T=0.5, erreur_limite=1):
    # liste pour stocker les w_separateurs au fil des itérations
    w_liste = []  
    
    # liste pour stocker les classifications au fil des itérations    
    classifications_liste = [classifications]
    
    # les vecteurs changés au cours des itérations 
    vecteurs_changes = [vecteurs * classifications[:, np.newaxis]]  # Produire un vecteur pondéré dès le début
    
    # première itération pour initialiser les tableaux
    w_sep_premier, _ = minimerror_l(vecteurs, classifications, w_initial, dimension_n, T=T, erreur_limite=erreur_limite)
    w_liste.append(w_sep_premier)
    
    # itérations pour calculer les hyperplans successifs
    for i in range(max_hyperplans):
        # calculer un nouvel hyperplan avec minimerror_l
        w_sep_iter, _ = minimerror_l(vecteurs_changes[i], classifications_liste[i], w_liste[i], dimension_n, T=T, erreur_limite=erreur_limite)
        w_liste.append(w_sep_iter)
        
        # reclasser les données avec le nouvel hyperplan
        nouvelles_classifications = classer_ensemble(vecteurs_changes[i], w_sep_iter)
        classifications_liste.append(nouvelles_classifications)
        
        # mettre à jour les vecteurs changés
        vecteurs_ponderes = vecteurs * nouvelles_classifications[:, np.newaxis]
        vecteurs_changes.append(vecteurs_ponderes)
    
    # retourner la liste des w_separateurs
    return w_liste


In [177]:
w_liste_xor = algorithme_monoplan(w_initial, xor_vectors, xor_classifications, 0.01)
w_liste_xor

Iteration 0: Erreur = 2.103909, Norme W = 1.0000
Iteration 100: Erreur = 1.934208, Norme W = 1.0000


Iteration 200: Erreur = 1.916372, Norme W = 1.0000
Iteration 300: Erreur = 1.894622, Norme W = 1.0000
Iteration 400: Erreur = 1.868462, Norme W = 1.0000
Iteration 500: Erreur = 1.837462, Norme W = 1.0000
Iteration 600: Erreur = 1.801305, Norme W = 1.0000
Iteration 700: Erreur = 1.759843, Norme W = 1.0000
Iteration 800: Erreur = 1.713142, Norme W = 1.0000
Iteration 900: Erreur = 1.661525, Norme W = 1.0000
Iteration 999: Erreur = 1.606179, Norme W = 1.0000
Nombre maximal d'itérations atteint.
[-0.74992891  0.46776416  0.46776416]
Iteration 0: Erreur = 1.386832, Norme W = 1.0000
Erreur limite atteinte après 9 itérations.
Erreur limite atteinte après 0 itérations.


[array([-0.74992891,  0.46776416,  0.46776416]),
 array([-0.27023716,  0.68079802,  0.68079802]),
 array([-0.27023716,  0.68079802,  0.68079802])]